In [1]:
from transformers import BartTokenizer, BartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq
import torch
from torch.utils.data import Dataset
import pandas as pd

/Users/shreyanakum/Downloads/Sophomore Year/Summer 2024/Aristocrat-Model/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_df = pd.read_csv('/Users/shreyanakum/Downloads/Sophomore Year/Summer 2024/Aristocrat-Model/data_collection/caesar_cipher_output.csv')

print(data_df.head())

# BART tokenizer
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

# Custom Dataset Class
class CaesarCipherDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=None):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        input_ids = self.tokenizer(item['Ciphertext'], padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt").input_ids.squeeze()
        labels = self.tokenizer(item['Plaintext'], padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt").input_ids.squeeze()
        
        return {'input_ids': input_ids, 'labels': labels}

# Set a max length for sequences --> change later to longer to see what happens
max_length = 128

# Create Dataset
dataset = CaesarCipherDataset(data_df, bart_tokenizer, max_length=max_length)

                                           Plaintext  \
0  His playing has taken him across the United Ki...   
1  In recent years Rino has developed his Conduct...   
2  Since arriving in Milan orchestrated Santa Mes...   
3  Recently at the Garden of Forgiveness, the two...   
4  Their shaded colonnades became busy markets on...   

                                          Ciphertext  
0  DEO LHWUEJC DWO PWGAJ DEI WYNKOO PDA QJEPAZ GE...  
1  EJ NAYAJP UAWNO NEJK DWO ZARAHKLAZ DEO YKJZQYP...  
2  OEJYA WNNEREJC EJ IEHWJ KNYDAOPNWPAZ OWJPW IAO...  
3  NAYAJPHU WP PDA CWNZAJ KB BKNCERAJAOO, PDA PSK...  
4  PDAEN ODWZAZ YKHKJJWZAO XAYWIA XQOU IWNGAPO KJ...  


/Users/shreyanakum/Downloads/Sophomore Year/Summer 2024/Aristocrat-Model/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
# Bart for sequence-to-sequence tasks
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
data_collator = DataCollatorForSeq2Seq(tokenizer=bart_tokenizer, model=model)

if torch.cuda.is_available():
    model.to('cuda')
    print(f"Using GPU: {torch.cuda.get_device_name(0)}") # for nvidia gpu in case its used
else:
    print("Using CPU")


training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir='./logs', 
    logging_strategy="steps",  # log every N steps
    logging_steps=10, # adjust to bigger
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
    data_collator=data_collator,
)

trainer.train()
trainer.save_model("./caesar_cipher_decoder")

Using CPU


/Users/shreyanakum/Downloads/Sophomore Year/Summer 2024/Aristocrat-Model/.venv/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/25428 [00:00<?, ?it/s]/Users/shreyanakum/Downloads/Sophomore Year/Summer 2024/Aristocrat-Model/.venv/lib/python3.11/site-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
  0%|          | 10/25428 [00:08<4:56:44,  1.43it/s]

{'loss': 11.4532, 'grad_norm': 26.63015365600586, 'learning_rate': 1.9992134654711343e-05, 'epoch': 0.0}


  0%|          | 20/25428 [00:14<4:41:29,  1.50it/s]

{'loss': 8.4901, 'grad_norm': 48.14887619018555, 'learning_rate': 1.9984269309422687e-05, 'epoch': 0.0}


...

{'loss': 0.079, 'grad_norm': 0.9199841022491455, 'learning_rate': 1.4157621519584712e-08, 'epoch': 3.0}


100%|█████████▉| 25420/25428 [5:11:54<00:04,  1.62it/s]

{'loss': 0.0873, 'grad_norm': 0.7286950945854187, 'learning_rate': 6.292276230926538e-09, 'epoch': 3.0}


100%|██████████| 25428/25428 [5:11:59<00:00,  1.73it/s]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
                                                       
100%|██████████| 25428/25428 [5:31:17<00:00,  1.28it/s]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 

{'eval_loss': 0.049213238060474396, 'eval_runtime': 1155.489, 'eval_samples_per_second': 58.68, 'eval_steps_per_second': 7.335, 'epoch': 3.0}
{'train_runtime': 19877.8826, 'train_samples_per_second': 10.233, 'train_steps_per_second': 1.279, 'train_loss': 0.19169289360884434, 'epoch': 3.0}


## Inference

In [4]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

# load the trained model and tokenizer
model = BartForConditionalGeneration.from_pretrained('./caesar_cipher_decoder')
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

# set to evaluation mode
model.eval()

# from df
ciphertext = "DEO LHWUEJC DWO PWGAJ DEI WYNKOO PDA QJEPAZ GEJCZKI LHWUEJC WP PDA HKJZKJ LWHHWZEQI WJZ PDA NKUWH WHXANP DWHH."
# HIS PLAYING HAS TAKEN HIM ACROSS THE UNITED KINGDOM PLAYING AT THE LONDON PALLADIUM AND THE ROYAL ALBERT HALL
input_ids = bart_tokenizer.encode(ciphertext, return_tensors='pt')

with torch.no_grad():
    generated_ids = model.generate(input_ids)

decoded_output = bart_tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(f"Ciphertext: {ciphertext}")
print(f"Decoded plaintext: {decoded_output}")


/Users/shreyanakum/Downloads/Sophomore Year/Summer 2024/Aristocrat-Model/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Users/shreyanakum/Downloads/Sophomore Year/Summer 2024/Aristocrat-Model/.venv/lib/python3.11/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Ciphertext: DEO LHWUEJC DWO PWGAJ DEI WYNKOO PDA QJEPAZ GEJCZKI LHWUEJC WP PDA HKJZKJ LWHHWZEQI WJZ PDA NKUWH WHXANP DWHH.
Decoded plaintext: His playing has taken him across the United Kingdom playing at the London Palladium and the
